In [ ]:
using Pkg
Pkg.activate(".")
Pkg.add("Distributions")
Pkg.instantiate()
Pkg.precompile()

In [4]:
using JuMP
using HiGHS
using Random

# Data

T = 1:24                    # time steps (example: 24 hours)

Q_data = fill(rand(Float64, (49315, 80137)), 24)     # Data center waste heat [kW] at each t
D_th = fill(800.0, 24)         # Greenhouse demand [kW] at each t
alpha = 1.5                     # Water evaporation factor [L/kWh rejected]

# Model

model = Model(HiGHS.Optimizer)

@variable(model, x_reuse[t in T] >= 0)              # Heat reused in CEA
@variable(model, x_reject[t in T] >= 0)             # heat rejected to cooling tower

# Objective
@objective(model, Min, sum(alpha * x_reject[t] for t in T))

# Constraints
@constraint(model, [t in T], x_reuse[t] + x_reject[t] == Q_data[t])
@constraint(model, [t in T], x_reuse[t] <= D_th[t])

optimize!(model)

println("Optimal total water use [L]: ", objective_value(model))
println("\nHour  x_reuse[kW]  x_reject[kW]")

for t in T
    println(rpad(t, 5), " ",
            rpad(value(x_reuse[t]), 12), " ",
            rpad(value(x_reject[t]), 12))
end


ErrorException: Subtraction between an array and a JuMP scalar is not supported: instead of `x - y`, do `x .- y` for element-wise subtraction.